In [7]:
theme = 'plotly_dark'
#theme = 'seaborn'

This k-means will find the min and max of data it's self and will quit once no points are to switch clusters and will find the "k" aka number of clusters by its self

It also plots the WCSS v. K  and plots the decrease amount v. K

This also uses k-means++ for better initialization

In [8]:
from math import floor
import plotly.express as px
import pandas as pd
import random
import centroid_2D as centroid_2D
import math
import plotly.graph_objects as go
import random


#genetrate input
data_random = []
data_cooked = []

N = 50
#random imput genarator size N  between 0 to 30
for i in range(N):
    x = random.randint(0, 30)
    y = random.randint(0, 30)
    cluster = 1
    data_random.append([x,y,cluster])

# NONrandom imput genarator size N  between 0 to 30
    
#this is for 4ish area example
p1 = floor(N/4) #first part of N
p2 = p1  #second part of N
p3 = p1 #third part
p4 = N - p1*3 # forth part 

for i in range(p1): 
    x = random.randint(2, 3)
    y = random.randint(1, 5)
    cluster = 1
    data_cooked.append([x,y, cluster])

for i in range(p2): 
    x = random.randint(25, 29)
    y = random.randint(15, 20)
    cluster = 1
    data_cooked.append([x,y, cluster])

for i in range(p3): 
    x = random.randint(14, 16)
    y = random.randint(16, 18)
    cluster = 1
    data_cooked.append([x,y, cluster])

for i in range(p4): 
    x = random.randint(0, 5)
    y = random.randint(25, 30)
    cluster = 1
    data_cooked.append([x,y, cluster])
'''
p1 = floor(N/4) #first part of N
p2 = p1  #second part of N
p3 = p1 #third part
p4 = N - p1*3 # forth part 

for i in range(p1): 
    x = random.randint(2, 8)
    y = random.randint(1, 15)
    cluster = 1
    data_cooked.append([x,y, cluster])

for i in range(p2): 
    x = random.randint(20, 29)
    y = random.randint(10, 20)
    cluster = 1
    data_cooked.append([x,y, cluster])

for i in range(p3): 
    x = random.randint(14, 16)
    y = random.randint(16, 30)
    cluster = 1
    data_cooked.append([x,y, cluster])

for i in range(p4): 
    x = random.randint(0, 12)
    y = random.randint(25, 30)
    cluster = 1
    data_cooked.append([x,y, cluster])
'''
'''
#this is for 2ish area example

h1 = floor(N/2) #first half of N
h2 = N-h1  #second half of N
for i in range(h1): 
    x = random.randint(15, 30)
    y = random.randint(10, 20)
    cluster = 1
    data_cooked.append([x,y, cluster])

for i in range(h2): 
    x = random.randint(0, 15)
    y = random.randint(0, 15)
    cluster = 1
    data_cooked.append([x,y, cluster])
'''

df_rand = pd.DataFrame(data_random)
df_cooked = pd.DataFrame(data_cooked)

df_rand =  df_rand.rename(columns={0:'X', 1:'Y', 2:'Cluster'})
df_cooked = df_cooked.rename(columns={0: 'X', 1: 'Y', 2:'Cluster'})

In [9]:
scatter_data = df_cooked
#scatter_data = df_rand

fig_scatter = px.scatter(scatter_data, x='X', y='Y', color='Cluster')
fig_scatter.update_layout(template=theme)
fig_scatter.update_layout(width=800, height=800)
fig_scatter.show()


In [10]:
def viz_data(data, centroids):
    data = pd.DataFrame(data)
    data = data.rename(columns={0: 'X', 1: 'Y', 2:'Cluster'})
    data["Cluster"] = data['Cluster'].astype(str)

    color_map = {'1': 'blue', '2': 'red','3': 'green','4': 'brown','5': 'orange','6': 'purple','7': 'pink','8': 'cyan','9': 'yellow','10': 'magenta'}

    fig = px.scatter(data, x='X', y='Y', color='Cluster', color_discrete_map=color_map, title="k-Means on Random Data")

    for c in centroids:
        num = str(c.cluster_number)
        color = color_map[num]
        fig.add_trace(go.Scatter(x=[c.x_old], y=[c.y_old], name="Centroid "+num, mode='markers', marker=dict(size=25, color=color)))
        fig.add_trace(go.Scatter(x=[c.x], y=[c.y], name="Next Spot for Centroid "+num, mode='markers', marker=dict(size=15, color='black', opacity=0.5)))

    print(data.head())
    fig.update_layout(template=theme)
    fig.update_layout(width=800, height=800)
    fig.show()

In [11]:
k_to_WCSS_dic = {}

'''
This code initializes k centers for k-means clustering according to a weighted probability distribution based on the squared distances distances(x).
'''
def k_means_pp_initialize(data, k):
    n = len(data)
    centers = []
    centers.append(random.choice(data))

    # Compute the squared distances to the nearest center for each data point
    for _ in range(1, k):
        distances = []
        for point in data:
            min_dist = float('inf')
            for c in centers:
                min_dist = min(min_dist, calc_distance(point[0]-c[0], point[1]-c[1]) ** 2)
            distances.append(min_dist)
        total_sum = sum(distances)
        probs = [x/total_sum for x in distances]
        centers.append(data[random.choices(range(n), weights=probs)[0]])
    
    return centers





def wcss(c):
    sum_of_squares = 0
    if len(c.coordinates) == 0:
        return 0
    for point in c.coordinates:
        d = calc_distance(point[0]-c.x, point[1]-c.y)
        sum_of_squares += d*d
    return sum_of_squares
    
#this will run k mean means for k to the  Max_k - 1
def unkown_means(data):
    MAX_k = 16
    for k in range(1, MAX_k):
        centroids = k_means(k,data)

        #these 2 lines are jsut for vizualiaion the data every iteration or just for some k
        if k == 3 or k == 4:
            viz_data(data=data,centroids=centroids)

        #now calc WCSS
        for c in centroids:
            k_to_WCSS_dic[k] = k_to_WCSS_dic.get(k, 0) + wcss(c)
        k_to_WCSS_dic[k] = k_to_WCSS_dic[k]
    
    print(k_to_WCSS_dic.keys())
    print(k_to_WCSS_dic.values())

    return centroids



def calc_distance(x,y):
    return math.sqrt((x*x) + (y*y))

def k_means(k, data):
    #create centriod
    centroids = [centroid_2D.Centroid() for _ in range(k)]



    initialized_clusters = k_means_pp_initialize(data, k)
    numbert = 1
    for c in centroids:
        c.cluster_number = numbert
        c.x = initialized_clusters[numbert-1][0]
        c.y = initialized_clusters[numbert-1][1]
        numbert += 1


    centroid_switched = True #if alreas on point switches centriod then this is true
    repetitions = 0 #we use this to make sure we are not stuck in infate loop of repetitions
    while centroid_switched and repetitions < 60:

        repetitions += 1
        centroid_switched = False

        #clearCentriods arrays
        for c in centroids:
            c.clear()  

        for point in data: #for each point find the centroid its closest to and add it to that 
            min_dist = float('inf')
            for c in centroids:
                new_dist = calc_distance(point[0]-c.x, point[1]-c.y)
                if new_dist < min_dist:
                    min_dist = new_dist
                    colosest_c = c       
            colosest_c.add_point(point[0], point[1])
            if colosest_c.cluster_number != point[2]: #this means it got to new centriod so we will wan to do at least one more iteration
                centroid_switched = True
            point[2] = colosest_c.cluster_number    #in the data change the centriod is assoside with

        for c in centroids:
            if len(c.coordinates) > 0:
                sum_x = 0
                sum_y = 0

                for point in c.coordinates:
                    sum_x += point[0]
                    sum_y += point[1]
                
                x_avg = sum_x/len(c.coordinates)
                y_avg = sum_y/len(c.coordinates)

                c.set_values(x_avg, y_avg)
    return centroids
    

In [12]:
#viz_data(data=data_random,centroids=unkown_means(data_random))

unkown_means(data_cooked)

ss_df = pd.DataFrame.from_dict(k_to_WCSS_dic, orient='index', columns=['WCSS'])
ss_df.index.name = 'K'

fig_ss = px.line(ss_df, x=ss_df.index, y='WCSS', title='WCSS vs. K')
fig_ss.update_layout(template=theme)
fig_ss.show()

ss_df['Rate_of_Decrease'] = ss_df['WCSS'].diff() / ss_df.index.to_series().diff()

fig_rate_of_decrease = px.scatter(ss_df, x=ss_df.index, y='Rate_of_Decrease', title='Rate_of_Decrease vs. K')
fig_rate_of_decrease.update_layout(template=theme)
fig_rate_of_decrease.show()


   X  Y Cluster
0  3  1       3
1  3  5       3
2  2  2       3
3  2  1       3
4  2  5       3


   X  Y Cluster
0  3  1       4
1  3  5       4
2  2  2       4
3  2  1       4
4  2  5       4


dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])
dict_values([9552.979999999996, 5143.266025641025, 1062.3273809523807, 167.11904761904762, 142.1904761904762, 108.86904761904762, 144.07460317460317, 72.6388888888889, 64.88888888888889, 78.48333333333333, 55.82738095238095, 39.95, 33.583333333333336, 34.275, 27.833333333333336])
